In [1]:
import requests
from bs4 import BeautifulSoup

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
r = requests.get('https://www.majcafe.com/volume-30/')

In [4]:
soup = BeautifulSoup(r.content)
a = soup.find_all('a')
pages = [a_.get('href') for a_ in a if 'volume-' in a_.get('href') or 'vol-' in a_.get('href')]
pages

['https://www.majcafe.com/volume-30/',
 'https://www.majcafe.com/vol-29-2022/',
 'https://www.majcafe.com/vol-28-2022/',
 'https://www.majcafe.com/vol-27-s1-2021/',
 'https://www.majcafe.com/vol-27-2021/',
 'https://www.majcafe.com/vol-26-2021/',
 'https://www.majcafe.com/vol-25-s1-2020/',
 'https://www.majcafe.com/vol-25-2020/',
 'https://www.majcafe.com/vol-24-s1-2020/',
 'https://www.majcafe.com/vol-24-2020/',
 'https://www.majcafe.com/vol-23-s1-2019/',
 'https://www.majcafe.com/vol-23-2019/',
 'https://www.majcafe.com/vol-22-s2-2019/',
 'https://www.majcafe.com/vol-22-s1-2019/',
 'https://www.majcafe.com/vol-22-2019/',
 'https://www.majcafe.com/vol-21-s1-2018/',
 'https://www.majcafe.com/vol-21-s2-2018/',
 'https://www.majcafe.com/vol-21-2018/',
 'https://www.majcafe.com/vol-20-s2-2017/',
 'https://www.majcafe.com/vol-20-s1-2017/',
 'https://www.majcafe.com/vol-20-2017/',
 'https://www.majcafe.com/volume-19-2016/',
 'https://www.majcafe.com/volume-18-2015/',
 'https://www.majcafe.c

In [7]:
from tqdm import tqdm

urls = []
for p in tqdm(pages):
    r = requests.get(p)
    soup = BeautifulSoup(r.content)
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href').endswith('.pdf')]
    urls.extend(a)
    
urls

100%|███████████████████████████████████████████| 41/41 [00:38<00:00,  1.08it/s]


['https://www.majcafe.com/wp-content/uploads/2023/06/MAJCAFE-VOL30-2023-v1.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/Paper-1-Vol-30-2023.pdf',
 'https://majcafe.com/wp-content/uploads/2022/11/PAPER2-MAJCAFE-VOL29a.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/PAPER-2.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/PAPER-3.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/PAPER-4.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/PAPER-5.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/PAPER-6.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/Paper-7-Vol-30-2023.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/PAPER-8.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/PAPER-9.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/PAPER-10.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/PAPER-11.pdf',
 'https://www.majcafe.com/wp-content/uploads/2023/07/PAPER-12.pdf',
 'https://w

In [8]:
urls_combined = sorted(list(set(urls)))
len(urls_combined)

490

In [9]:
!mkdir majcafe.com

In [10]:
import os

pdfs = urls_combined
for i in tqdm(range(len(pdfs))):
    filename = os.path.join('majcafe.com', f'{i}.pdf')
    if os.path.exists(filename):
        continue
        
    u = pdfs[i]
        
    r = requests.get(u)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

100%|█████████████████████████████████████████| 490/490 [25:42<00:00,  3.15s/it]


In [11]:
from glob import glob
from tika import parser
import json

TIKA_HOST = 'http://localhost:9998'

In [12]:
files = glob('majcafe.com/*.pdf')

with open('majcafe.com.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
            body = BeautifulSoup(raw_xml['content']).find('body')
            t = '\n'.join(body.findAll(text=True))
            fopen.write(f'{json.dumps(t)}\n')
        except:
            pass

100%|█████████████████████████████████████████| 490/490 [00:25<00:00, 19.55it/s]
